<a href="https://colab.research.google.com/github/Mertkuludag/Colab_Crop_Type/blob/main/Ankara_Crop_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importlar

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached geemap-0.13.7-py2.py3-none-any.whl (2.0 MB)
  Using cached folium-0.12.1.post1-py2.py3-none-any.whl (95 kB)
  Using cached python_box-6.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached xyzservices-2022.4.0-py3-none-any.whl (36 kB)
  Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
  Using cached PyCRS-1.0.2-py3-none-any.whl
  Using cached ipyevents-2.0.1-py2.py3-none-any.whl (130 kB)
  Using cached geeadd-0.5.5-py3-none-any.whl (30 kB)
  Using cached mapclassify-2.4.3-py3-none-any.whl (38 kB)
  Using cached bqplot-0.12.33-py2.py3-none-any.whl (1.2 MB)
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached whiteboxgui-0.7.0-py2.py3-none-any.whl (99 kB)
  Using cached ee_extra-0.0.13-py3-none-any.whl
  Using cached pyshp-2.3.0-py2.py3-none-any.whl (46 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
 

In [2]:
# Installs geemap package
import google.colab

import geemap

# Authenticates and initializes Earth Engine
import ee

ee.Authenticate()
ee.Initialize()  

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OiOMViszMEdD04ytGkniFuzV1tmNCu6Q-tHhD0JSNkQ&tc=DpLxJec7jxGytEEXDGGcnyoO_MWqpxokiznL-s6eM1U&cc=RCFDleBa3sXiGCAaP4jNtDFmbCVOds9wehTDnIEGlkM

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgUhdoL6oXHj7_FlifgwDusT7IHEz3opJVseMtJsks58a7epdfjvEM

Successfully saved authorization token.


---

## Görüntü Maskeleme

In [19]:
Ankara = ee.FeatureCollection("users/mkuludag93/Sinirlar/Ankara_Sinir");
Ankara_Parsel = ee.FeatureCollection("users/mkuludag93/Ankara_parseller_tamami");

roi = Ankara.geometry()

# Bulut Maskesi
def se2mask(image):
    qa = image.select('QA60')

    mask = qa.bitwiseAnd(1<<10).Or(qa.bitwiseAnd(1<<11))

    return image.updateMask(mask.Not())

In [20]:
# Gece Görüntüsü ile Kent alanları maskesi

nightset = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2021-01-01","2021-11-28")


nightsetmed = nightset.median().clip(Ankara) 
avg_rad = nightsetmed.select('avg_rad')

def nightmask(img):
  nightexc = avg_rad.gte(7)
  img = img.updateMask(nightexc.unmask().Not())

  return(img);   

In [21]:
# Su kütlesi için maskeleme

gsw = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
occurrence = gsw.select('occurrence')

def watermask(img):
  excludewater = occurrence.gte(5); 
  img = img.updateMask(excludewater.unmask().Not())
  return(img)

---

## Görüntüleri Ekleme ve Composite

### Sentinel 2

In [25]:
# Çalışma alanımızı ve Yükseklik verisini tanımlayalım
s2bands = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":3000,"bands":s2bands}

def sentinel2goruntu(tarih1, tarih2):
    se = ee.ImageCollection('COPERNICUS/S2').filterDate(
    tarih1,tarih2).filterBounds(Ankara).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).map(nightmask).map(watermask).median()

    se_parsel = se.select(s2bands).clip(Ankara_Parsel)
 
    ndvi = se.normalizedDifference(['B8', 'B4']).rename([f'ndvi_{tarih1}']);
    ndbi = se.normalizedDifference(['B11', 'B8']).rename([f'ndbi_{tarih1}']);

    return se_parsel.addBands(ndvi).addBands(ndbi)


se1 = sentinel2goruntu("2021-05-15", "2021-05-29")
se2 = sentinel2goruntu("2021-06-15", "2021-06-29")
se3 = sentinel2goruntu("2021-07-15", "2021-07-29")
se4 = sentinel2goruntu("2021-08-15", "2021-08-29")

fusedse2 = se1.addBands(se2).addBands(se3).addBands(se4)






### Sentinel 1

In [26]:



def sentinel1goruntu(tarih1, tarih2):
  se = ee.ImageCollection('COPERNICUS/S1_GRD').filter(ee.Filter.date(tarih1, tarih2))\
  .filterBounds(roi)\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
  .filter(ee.Filter.eq('instrumentMode', 'IW'))\

  desc = se.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
  asc = se.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));

  composites1 = ee.Image.cat([
    asc.select('VH').mean(),
    asc.select('VV').mean(),
    desc.select('VH').mean()
  ]).focal_median();

  composite_1 = composites1.clip(Ankara)
  composite_1= nightmask(composite_1)
  composite_1 =watermask(composite_1)
  composite_1= composite_1.clip(Ankara_Parsel)

  return composite_1


se1_1= sentinel1goruntu("2021-05-15", "2021-05-29")
se1_2= sentinel1goruntu("2021-06-15", "2021-06-29")
se1_3= sentinel1goruntu("2021-07-15", "2021-07-29")
se1_4= sentinel1goruntu("2021-08-15", "2021-08-29")

fusedse1 = se1_1.addBands(se1_2).addBands(se1_3).addBands(se1_4)

finalfuse = fusedse2.addBands(fusedse1)

# Sentinel 1 ve 2 nin composite edilmesi
print(finalfuse.bandNames().getInfo())

['B4', 'B3', 'B2', 'ndvi_2021-05-15', 'ndbi_2021-05-15', 'B4_1', 'B3_1', 'B2_1', 'ndvi_2021-06-15', 'ndbi_2021-06-15', 'B4_2', 'B3_2', 'B2_2', 'ndvi_2021-07-15', 'ndbi_2021-07-15', 'B4_3', 'B3_3', 'B2_3', 'ndvi_2021-08-15', 'ndbi_2021-08-15', 'VH', 'VV', 'VH_1', 'VH_2', 'VV_1', 'VH_1_1', 'VH_3', 'VV_2', 'VH_1_2', 'VH_4', 'VV_3', 'VH_1_3']


---

 ##  Sınıflama

In [43]:
# Sınıflar
bugday = ee.FeatureCollection("users/mkuludag93/Tarim_Parselleri/Ankara_Bugday_Training")
misir = ee.FeatureCollection("users/mkuludag93/Tarim_Parselleri/Ankara_Misir_Training")
aycicegi = ee.FeatureCollection("users/mkuludag93/Tarim_Parselleri/Ankara_Aycicegi_Training")

# Train setimizi kombinledik

trainingfc = bugday. merge(aycicegi) .merge(misir);
label = 'landcover';


trainingGcp = trainingfc



# Noktaları SampleRegionladık (Böylece örnek noktaların bulunduğu pixellerin orta noktalarını alarak her bir banddan gelen verileri elde ettik).
training = finalfuse.sampleRegions(**{
  "collection": trainingGcp,
  "properties": ['landcover'],
  "scale": 10,
  "tileScale": 16
});


# Eğitim Vakti!!
classifier = ee.Classifier.smileRandomForest(50).train(**{
  "features": training,  
  "classProperty": 'landcover',
  "inputProperties": finalfuse.bandNames()
});


classified = finalfuse.classify(classifier).clip(Ankara_Parsel);




palette = [ 
  'FAFA28', #(0)  Buğday           
  '990000', #(1)  misir   
  '7DB842', #(2)  Aycicegi 
];  

legend_keys = ['Buğday', 'Misir', 'Ayçiçeği']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['FAFA28', '990000', '7DB842']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]



### Görüntüyü Haritada Görüntüleme

In [ ]:
# Haritamıza sentinel 2'yi ekleyelim
# Görütüleme parametrelerini tanımlayalım
s2bands = ['B4','B3','B2']
rgbViz = {"min":0.0, "max":3000,"bands":s2bands}


map2 = geemap.Map()
map2.add_basemap('HYBRID')
map2.centerObject(Ankara, 7)

map2.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)


# map2.addLayer(fusedse1, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'Sentinel 1');
map2.addLayer(finalfuse, rgbViz, 'Sentinel Tümü Görüntüsü 2021');
map2.addLayer(trainingGcp, {}, 'training', False)

map2.addLayer(classified, {min: 0, max: 2, palette: palette}, '2021 Sınıflanmış Görüntü');

map2.addLayerControl()
map2


---

## Doğruluk Analizi

In [ ]:
# Train Accur
train_accuracy = classifier.confusionMatrix()
print('Train Accuracy', train_accuracy.accuracy().getInfo())
train_accuracy.producersAccuracy().getInfo()
train_accuracy.consumersAccuracy().getInfo()


# Test Accur
validated = validationGcp.classify(classifier)

testConfusionMatrix = validated.errorMatrix('landcover', 'classification')

print('Test Confusion Matrix', testConfusionMatrix);
print('Test Accuracy', testConfusionMatrix.accuracy());
print('Test Consumers Accuracy:', testConfusionMatrix.consumersAccuracy()); 
print('Test Producers Accuracy:', testConfusionMatrix.producersAccuracy()); 


---

## Sonuçları Export Etme


In [ ]:
# Exporting Asset

task = ee.batch.Export.image.toAsset(**{
  'image': se2_parsel,
  'description': 'Ankara_Parseller',
  'assetId': 'users/mkuludag93/Ankara_Parseller',
  'scale': 20,
  'region': Ankara.geometry()
})
task.start()